In [1]:
import pandas as pd
import re
from collections import Counter

In [2]:
train_df = pd.read_pickle("../../data/train_IOB_repl_compound.pkl")
test_df = pd.read_pickle("../../data/test_IOB_repl_compound.pkl")

In [23]:
def freq_headlines(df, n=None, _min=None):
    h_list = []
    for i, h in df.loc[:, ['h2']].iteritems():
        h_list += h.tolist()

    h_count = Counter(h_list)
    
    if n:
        return [h for h, _ in h_count.most_common(n + 1)]
    elif _min:
        return [h for h, count in h_count.most_common() if count >= _min]

def assign_cat_headline(df, headlines, col):
    '''
    Other headline         = 0
    Contains freq headline = 1
    No headline            = 2
    '''
    df[col] = 0

    df.loc[
        df.loc[:, ['h2']].fillna('NO_TITLE').apply(
            lambda x: x.str.match('|'.join([re.escape(h) for h in headlines]))
        ).sum(1) > 0
        , col
    ] = 1

    df.loc[
        df.loc[:, ['h2']].isna().all(1)
        , col
    ] = 2
    
    return df

In [24]:
production_headlines = freq_headlines(train_df.loc[train_df.production_tag_seq.apply(lambda x: 'B' in x)], _min=5)

train_df = assign_cat_headline(train_df, headlines=production_headlines, col='cat_production_headline')
test_df = assign_cat_headline(test_df, headlines=production_headlines, col='cat_production_headline')

In [25]:
raw_material_headlines = freq_headlines(train_df.loc[train_df.raw_material_tag_seq.apply(lambda x: 'B' in x)], _min=5)

train_df = assign_cat_headline(train_df, headlines=raw_material_headlines, col='cat_raw_material_headline')
test_df = assign_cat_headline(test_df, headlines=raw_material_headlines, col='cat_raw_material_headline')

In [26]:
train_df.to_pickle("../../data/train_IOB_repl_compound.pkl")
test_df.to_pickle("../../data/test_IOB_repl_compound.pkl")

In [27]:
test_df.groupby('cat_raw_material_headline').count()

,_id,label,sentence,title,words,repl_words,furigana_tag_seq,another_name_tag_seq,use_tag_seq,type_tag_seq,...,density_tag_seq,melting_tag_seq,boiling_tag_seq,rational_formula_tag_seq,h2,h3,h4,h5,h6,cat_production_headline
cat_raw_material_headline,,,,,,,,,,,,,,,,,,,,,
0,633,633,633,633,633,633,633,633,633,633,...,633,633,633,633,633,142,0,0,0,633
1,492,492,492,492,492,492,492,492,492,492,...,492,492,492,492,492,142,0,0,0,492
2,439,439,439,439,439,439,439,439,439,439,...,439,439,439,439,0,1,0,0,0,439
